In [3]:
import pandas as pd
import numpy as np
import mlflow
import mlflow.sklearn
import mlflow.xgboost
from sklearn.model_selection import train_test_split, RandomizedSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from xgboost import XGBClassifier
from sklearn.metrics import roc_auc_score, classification_report
from imblearn.over_sampling import SMOTE
import joblib
import time

# ==========================================
# 1. SETUP & DATA LOADING
# ==========================================
mlflow.set_tracking_uri("http://localhost:5000")
mlflow.set_experiment("Gobest_Cab_Safety_Prediction")

print("Loading Data...")
df = pd.read_csv("../data/bi_dataset.csv")

# Basic Cleanup (Dropping IDs and Leaks only)
drop_cols = ['bookingID', 'driver_id', 'date_of_birth', 'name', 
             'car_brand', 'car_model_year', 'rating_bin', 'exp_bin', 
             'gender', 'is_dangerous', 'label']
X = df.drop(columns=[c for c in drop_cols if c in df.columns]).fillna(0)
y = df['label']

# Split & SMOTE
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)
smote = SMOTE(random_state=42)
X_train_res, y_train_res = smote.fit_resample(X_train, y_train)

print(f"Data Ready. Training Features: {X.shape[1]}")

# ==========================================
# 2. DEFINE THE COMPETITORS
# ==========================================

# A. Logistic Regression Grid
log_reg = LogisticRegression(solver='lbfgs', max_iter=1000)
log_param_dist = {
    'C': [0.01, 0.1, 1, 10, 100],
    'penalty': ['l2'] # lbfgs supports l2
}

# B. Random Forest Grid
rf = RandomForestClassifier(random_state=42)
rf_param_dist = {
    'n_estimators': [100, 200, 300],
    'max_depth': [10, 20, 30, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'bootstrap': [True, False]
}

# C. XGBoost Grid
xgb = XGBClassifier(objective='binary:logistic', eval_metric='auc', random_state=42)
xgb_param_dist = {
    'n_estimators': [100, 200, 300, 500],
    'max_depth': [3, 6, 10, 15],
    'learning_rate': [0.01, 0.05, 0.1, 0.2],
    'subsample': [0.7, 0.8, 0.9, 1.0],
    'colsample_bytree': [0.7, 0.8, 1.0]
}

# Store them in a dictionary for looping
competitors = [
    ("Logistic_Regression_V1", log_reg, log_param_dist),
    ("Random_Forest_V1", rf, rf_param_dist),
    ("XGBoost_V1", xgb, xgb_param_dist)
]

# ==========================================
# 3. THE GRAND TOURNAMENT (Tune All)
# ==========================================

for name, model, param_dist in competitors:
    print(f"\n>>> Starting Tuning for: {name}")
    start_time = time.time()
    
    with mlflow.start_run(run_name=f"Tune_{name}"):
        # RandomizedSearch (Tuning)
        search = RandomizedSearchCV(
            estimator=model,
            param_distributions=param_dist,
            n_iter=20,          # Try 20 random combinations for EACH model
            scoring='roc_auc',
            cv=3,
            verbose=1,
            random_state=42,
            n_jobs=-1
        )
        
        search.fit(X_train_res, y_train_res)
        
        # Get Best Result
        best_model = search.best_estimator_
        best_params = search.best_params_
        best_auc = search.best_score_ # CV Score
        
        # Evaluate on Test Set (The Real Test)
        if "XGBoost" in name:
            y_prob = best_model.predict_proba(X_test)[:, 1]
        else:
            y_prob = best_model.predict_proba(X_test)[:, 1]
            
        final_auc = roc_auc_score(y_test, y_prob)
        
        print(f"   Best Params: {best_params}")
        print(f"   Test Set AUC: {final_auc:.4f}")
        
        # Log to MLflow
        mlflow.log_params(best_params)
        mlflow.log_metric("auc", final_auc)
        
        if "XGBoost" in name:
            mlflow.xgboost.log_model(best_model, "model")
        else:
            mlflow.sklearn.log_model(best_model, "model")
            
        print(f"   Finished {name} in {time.time() - start_time:.1f}s")

Loading Data...
Data Ready. Training Features: 28

>>> Starting Tuning for: Logistic_Regression_V1
Fitting 3 folds for each of 5 candidates, totalling 15 fits


c:\Users\Thomas\Desktop\PAI CA2\gobest-cab-safety-sprint2\CA2_Sprint2\venv\Lib\site-packages\sklearn\model_selection\_search.py:324: UserWarning: The total space of parameters 5 is smaller than n_iter=20. Running 5 iterations. For exhaustive searches, use GridSearchCV.
  warnings.warn(
c:\Users\Thomas\Desktop\PAI CA2\gobest-cab-safety-sprint2\CA2_Sprint2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:1135: FutureWarning: 'penalty' was deprecated in version 1.8 and will be removed in 1.10. To avoid this warning, leave 'penalty' set to its default value and use 'l1_ratio' or 'C' instead. Use l1_ratio=0 instead of penalty='l2', l1_ratio=1 instead of penalty='l1', and C=np.inf instead of penalty=None.
  warnings.warn(
c:\Users\Thomas\Desktop\PAI CA2\gobest-cab-safety-sprint2\CA2_Sprint2\venv\Lib\site-packages\sklearn\linear_model\_logistic.py:406: ConvergenceWarning: lbfgs failed to converge after 1000 iteration(s) (status=1):
STOP: TOTAL NO. OF ITERATIONS REACHED LIMIT

Increase

   Best Params: {'penalty': 'l2', 'C': 1}
   Test Set AUC: 0.7279


2026/01/18 21:31:30 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   Finished Logistic_Regression_V1 in 15.1s
🏃 View run Tune_Logistic_Regression_V1 at: http://localhost:5000/#/experiments/671472067514299670/runs/92c0406e89874dd796461cc0fd9e8c30
🧪 View experiment at: http://localhost:5000/#/experiments/671472067514299670

>>> Starting Tuning for: Random_Forest_V1
Fitting 3 folds for each of 20 candidates, totalling 60 fits
   Best Params: {'n_estimators': 200, 'min_samples_split': 2, 'min_samples_leaf': 1, 'max_depth': None, 'bootstrap': False}
   Test Set AUC: 0.6945


2026/01/18 21:33:29 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   Finished Random_Forest_V1 in 118.9s
🏃 View run Tune_Random_Forest_V1 at: http://localhost:5000/#/experiments/671472067514299670/runs/ca0861f26f80416c9a2cd911c50af4e2
🧪 View experiment at: http://localhost:5000/#/experiments/671472067514299670

>>> Starting Tuning for: XGBoost_V1
Fitting 3 folds for each of 20 candidates, totalling 60 fits
   Best Params: {'subsample': 0.7, 'n_estimators': 300, 'max_depth': 15, 'learning_rate': 0.2, 'colsample_bytree': 1.0}
   Test Set AUC: 0.6919


2026/01/18 21:34:22 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.


   Finished XGBoost_V1 in 54.0s
🏃 View run Tune_XGBoost_V1 at: http://localhost:5000/#/experiments/671472067514299670/runs/cd9ef5f4f98c4787812e47a787ae5930
🧪 View experiment at: http://localhost:5000/#/experiments/671472067514299670
